In [1]:
#load libraries
import ee
import datetime
import fiona
from shapely.geometry import shape
import itertools
import os.path
import time
import os

In [2]:
#set important variables that probably do change from time to time
extent_shapefile_path = '/home/dan/Documents/react_data/Cities_React/Boundaries.shp'
product = 'NIGHTTIME_LIGHTS'
modis_product =  'NOAA/DMSP-OLS/' + product #'MODIS/006/' + product
unmaskval = -9999
start_year = 1992
end_year = 2014
end_year = end_year + 1 #because how range works
variables = ['avg_vis', 'stable_lights', 'cf_cvg', 'avg_lights_x_pct'] # 'QC_Day', 'LST_Night_1km', 'QC_Night']
target_crs = 'SR-ORG:6974'
redl = True
max_px_number = 1200000000

In [3]:
#load the shapefile
boundaries = fiona.open(extent_shapefile_path)
the_cities = list(boundaries.items())

In [4]:
if (extent_shapefile_path == '/home/dan/Documents/react_data/Cities_React/roi_1.shp'):
    the_cities[0][1]['properties']['Name'] = 'roi_1'

In [5]:
#get a list of the city names
citynames = [item[1]['properties']['Name'] for item in the_cities]

In [6]:
#create a grid/dictionary of things to download
listolists = [variables, citynames, range(start_year,end_year)]
combos = list(itertools.product(*listolists))

In [7]:
#find which files are already downloaded
dlpath = '/modis/'
filenames = [dlpath + c[1].replace(' ','') + "_" + product +'_'+c[0]+'_'+str(c[2]) +'.tif'  for c in combos]
#already_downloaded = [os.path.isfile(fff) for fff in filenames]

In [10]:
listolists = [['avg_vis'], ['Ouagadougou'], [1994]]
combos = list(itertools.product(*listolists))

In [11]:
#Start earth engine
ee.Initialize()
#pull the product
imgs = ee.ImageCollection(modis_product)

In [12]:
#for each possible combo
counter = 0
for ccc in range(0, len(combos)):
    #check to see if the file is already downloaded
    if (True): #(filenames[ccc] in gdrivefilelist) == False:
        year = combos[ccc][2]
        city_name_space = combos[ccc][1]
        city_name = city_name_space.replace(' ','')
        vari = combos[ccc][0]
        
        #sort out the year
        start = str(year)+'-'+'01-01'
        end = str(year+1)+'-'+'01-01' #turns out, its exclusive
        dat = imgs.filterDate(start, end).select(vari).toList(1000)

        #find the bounding box of the city
        for cit in range(0,len(the_cities)):
            if city_name_space == the_cities[cit][1]['properties']['Name']:
                bbox = shape(the_cities[cit][1]['geometry']).bounds
            

        city_name = city_name.replace(' ','')

        #name the job
        job_id = city_name + "_" + product +'_'+vari+'_'+str(year)

        result = ee.Image.cat(dat.get(0))

        #create a polygon to clip things too
        # xMin, yMin, xMax, yMax
        target_projection = ee.Projection(target_crs)
        thecity = ee.Geometry.Rectangle(bbox) #.transform(target_projection,250)
        
        if (dat.length()>1):
            for i in range(1,dat.length().getInfo()):
                newimage = ee.Image.cat(dat.get(i))
                result = ee.Image.cat([result, newimage])
        
        ##unmask the image
        result = result.unmask(unmaskval)
        
        #request to download
        #https://stackoverflow.com/questions/39219705/how-to-download-images-using-google-earth-engines-python-api
        newfolder = 'earth_engine/' + product + '/' + city_name
        task = ee.batch.Export.image.toDrive(image = result, folder = 'modis', 
                                             description = job_id, fileNamePrefix = job_id,
                                             region = thecity.getInfo().get('coordinates'),
                                             skipEmptyTiles=False,
                                             maxPixels = max_px_number)
        #print(job_id)
        task.start()
        
        
        
        if (counter +1) % 100 == 0:
            time.sleep(100)
        
        counter = counter + 1

In [21]:
dat.getInfo()[1]

{u'bands': [{u'crs': u'EPSG:4326',
   u'crs_transform': [0.0083333333,
    0.0,
    -180.00416666665,
    0.0,
    -0.0083333333,
    75.00416666665],
   u'data_type': {u'max': 255,
    u'min': 0,
    u'precision': u'int',
    u'type': u'PixelType'},
   u'dimensions': [43201, 16801],
   u'id': u'avg_vis'}],
 u'id': u'NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F121994',
 u'properties': {u'system:asset_size': 738243997,
  u'system:index': u'F121994',
  u'system:time_end': 788918400000,
  u'system:time_start': 757382400000},
 u'type': u'Image',
 u'version': 1504844053594062}

In [ ]:
dat = imgs.filterDate(start, end)

In [ ]:
info = dat.getInfo().get('features')
for i in range(0, len(info)):
    print info[i].get('id')

In [ ]:
newfolder = 'earth_engine/' + product + '/' + city_name


In [ ]:
newfolder
newfolder = 'earth_engine/test/'

In [ ]:
task = ee.batch.Export.image.toDrive(image = result, folder = 'modis', 
                                             description = job_id, fileNamePrefix = job_id,
                                             region = thecity.getInfo().get('coordinates'),
                                             skipEmptyTiles=False,
                                             maxPixels = max_px_number)
#print(job_id)
task.start()

In [ ]:
len(combos
   )